In [1]:
import numpy as np
import gradflow.comp_graph as cg

import autograd as ag

In [2]:
d_in = 7
d_out = 1
d_hid = 13

w_A_np = np.random.normal(size=(d_hid, d_in))
w_C_np = np.random.normal(size=(d_out, d_hid))
x_np = np.random.normal(size=(d_in, 1))

x = cg.Value("x", x_np)

w_A = cg.Value("w_A", w_A_np)
linear1 = cg.Dot("linear1")

act1 = cg.Tanh("act1")

w_C = cg.Value("w_C", w_C_np)
linear2 = cg.Dot("linear2")

# we use calls to forward pass to define the function topology
x.forward()
w_A.forward()
w_C.forward()
y = linear1.forward(w_A, x)
y = act1.forward(linear1)
y = linear2.forward(w_C, act1)

In [3]:
def test_function(input):
    z = ag.numpy.dot(w_A_np, input)
    z = ag.numpy.tanh(z)
    z = ag.numpy.dot(w_C_np, z)
    return z

In [4]:
atol = 1e-5

y_ag = test_function(x_np)
assert np.allclose(y, y_ag, atol=atol)

In [ ]:
# compute backward pass by going in reverse topological order
# doing this manually but this should be replaced by a topo sort
# linear2.d_out = np.array([[1.]])
# _ = linear2.backward()
# _ = act1.backward()
# _ = linear1.backward()


my_graph = cg.Graph("my_fun", [linear2])
my_graph.backward()

In [ ]:
dy_dx_ag = ag.grad(test_function)(x_np)

assert np.allclose(x.d_out, dy_dx_ag, atol=atol)